## Constructing a quantum adder and quantum multiplier Circuit 

Sources: [1]  Quantum arithmetic with the Quantum Fourier Transform - https://arxiv.org/pdf/1411.5949.pdf <br>
  &emsp;&emsp;&emsp;&emsp; [2] Addition on a Quantum Computer - https://arxiv.org/pdf/quant-ph/0008033.pdf <br>
  &emsp;&emsp;&emsp;&emsp; [3] IBM Qiskit - https://qiskit.org/textbook/preface.html <br>
  &emsp;&emsp;&emsp;&emsp; [4] M. A. Nielsen and I. L. Chuang, Quantum Computation and Quantum Information: 10th Anniversary
Edition. Cambridge University Press, 2010.

In [34]:
# Importing Libraries

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import math
import numpy as np
import time

In [3]:
# Defining some necessary function

def to_binary(n):           # converts upto 9-bit binary, intentionally kept 9 bits so we could include big numbers
    return format(n,'09b')

# Initializing qubits according to binary representation of number,
# m is from where it has to start to put binary representation in qubits
def initializing_qubits(qc,binary_n,m):
    if m<len(binary_n):
        for j in range(m,m+len(binary_n)):
            if binary_n[j-m] == '1':  # Applies an NOT gate for bit where the binary representation has 1
                qc.x(j)
    else:
        for j in range(m,m+len(binary_n)):
            if binary_n[j-len(binary_n)-m] == '1': # Applies an NOT gate for bit where the binary representation has 1
                qc.x(j)
    return qc

# Quantum Fourier Transform 
def qft(qc,n):            # Applies Quantum Fourier transform on given circuit provided to first n-qubits
    for i in range(n):
        qc.h(i)
        for j in range(i+1,n):
            qc.cp(np.pi/(2**(j-i)),j,i)
    for i in range(n//2):
        qc.swap(i, n-i-1)
    return qc


# Inverse Quantum Fourier Transform 
def iqft(qc,n):             # Applies Inverse Quantum Fourier transform on  given circuit provided to first n-qubits
    for i in range(n):
        qc.h(i)
        for j in range(i+1,n):
            qc.cp(-np.pi/(2**(j-i)),j,i)
    for i in range(n//2):
        qc.swap(i, n-i-1)
    return qc  


In [61]:
#-----------------------------------------------------Adder---------------------------------------------------------#

# The numbers to be added should be of same number of bits, we are using upto 9 bits but it could easily be changed
# to include even bigger numbers by changing just a number in '09b' to for ex. '08b' in to_binary function

def adder(n1,n2):
    
    binary_n1 = to_binary(n1)  # Converting to binary
    #binary_n1 = binary_n1[::-1]

    binary_n2 = to_binary(n2)  # Converting to binary
    #binary_n2 = binary_n2[::-1]

    qc = QuantumCircuit(len(binary_n1)+len(binary_n2)+1,len(binary_n1)+1)        

    # Putting binary representation of numbers given in qubits
    qc = initializing_qubits(qc,binary_n1,1) 
    qc = initializing_qubits(qc,binary_n2,len(binary_n1)+1)

    qc = qft(qc,len(binary_n1)+1) # Taking QFT of n1+1 qubits because sum of two 4-bit number might be a 5-bit number


    # These are the rotations required which moves the qubits for QFT of first number n1 in such a way that 
    # the rotations on bloch sphere are at angle where if we take inverse QFT we get (n1+n2)
#-------------------------------------------------Starts here-------------------------------------------------------#
  
    for i in range(len(binary_n1)+1,len(binary_n1)+len(binary_n2)+1):
        qc.cp(np.pi/2**(i-len(binary_n1)),i,len(binary_n1))
    
    for j in range(1,len(binary_n2)+1):
        for i in range(len(binary_n1)+j,len(binary_n1)+len(binary_n2)+1):
            qc.cp(np.pi/2**(i-len(binary_n1)-j),i,len(binary_n1)-j)
        
#-------------------------------------------------Ends here---------------------------------------------------------#

    qc = iqft(qc,len(binary_n1)+1)            #Taking inverse QFT

    for i in range(len(binary_n1)+1):         # Measuring the qubits where the answer gets stored
        qc.measure(i,i)
    
    #qc.draw('mpl')
    
    sim = Aer.get_backend('aer_simulator')
    counts = sim.run(assemble(qc)).result().get_counts()
    #plot_histogram(counts)
    #print(counts)
    
    # Now we need to convert the answer which is in binary back to decimal
    ans = []
    for i in reversed(range(2,len(binary_n1)+3)):
        ans.append(format(counts)[i])
    
    ans = ''.join(ans)
    ans = int(ans,2)
    depth = qc.depth()
    
    return ans,depth

n1 = 3
n2 = 60
print("n1 =",n1)
print("n2 =",n2)
start = time.time()
print("n1+n2 =",adder(n1,n2)[0])
end = time.time()
print("depth of the adder circuit =",adder(n1,n2)[1]) # depth could be decreased by taking less number of qubits       
print(f"Time for execution of the adder circuit = {(end-start):.2f} seconds")

n1 = 3
n2 = 60
n1+n2 = 63
depth of the adder circuit = 59
Time for execution of the adder circuit = 0.15 seconds


In [63]:
#---------------------------------------------- Multiplier ---------------------------------------------------------#

# What we are doing here is repeating n1+n1, (n2-1) times if we have n1*n2

def multiplier(n1,n2):
    ans = 0
    depth = 0
    if n1>n2:
        n1,n2 = n2,n1          # this is so that adder is used the number of times we have smaller number
    for i in range(n1):
        ans = adder(ans,n2)[0]
        depth = depth + adder(ans,n2)[1]
    return ans,depth

n1 = 6
n2 = 5
print("n1 =",n1)
print("n2 =",n2)
start = time.time()
print("n1*n2 =",multiplier(n1,n2)[0]) 
end = time.time()
print("depth of the multiplier circuit for given numbers =",multiplier(n1,n2)[1])  # The variation here is due to how many times the loop runs
print(f"Time for execution of the Multiplier circuit = {(end-start):.2f} seconds")

n1 = 6
n2 = 5
n1*n2 = 30
depth of the multiplier circuit for given numbers = 295
Time for execution of the Multiplier circuit = 1.25 seconds
